In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/siddhanthmate/Desktop/AllFiles/CODE/WORK_CODE/fintech/DATA/BANKNIFTY/01-06-2023.csv")

In [4]:
df.head()

,Ticker,Date,Time,Open,High,Low,Close,Volume,OI,Type,Script,Strike,Expiry,Contract_Monthly,Contract_Weekly
0,BANKNIFTY01JUN2336500PE.NFO,2023-04-21,10:20:59,34.55,34.55,34.55,34.55,25,0,PE,BANKNIFTY,36500,2023-06-01,W,M
1,BANKNIFTY01JUN2336500PE.NFO,2023-04-21,10:59:59,34.55,34.55,34.55,34.55,25,25,PE,BANKNIFTY,36500,2023-06-01,W,M
2,BANKNIFTY01JUN2338000PE.NFO,2023-04-21,11:37:59,122.00,122.00,122.00,122.00,25,0,PE,BANKNIFTY,38000,2023-06-01,W,M
3,BANKNIFTY01JUN2342000PE.NFO,2023-04-21,12:32:59,688.95,688.95,680.90,680.90,1575,1475,PE,BANKNIFTY,42000,2023-06-01,W,M
4,BANKNIFTY01JUN2342000PE.NFO,2023-04-21,12:33:59,674.85,674.85,674.85,674.85,100,1475,PE,BANKNIFTY,42000,2023-06-01,W,M


In [5]:
import re
import threading
import pymongo
import csv
import os

client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['MARKET_DATA']
collection = db['SPOT_DATA']

pattern = r'^(.+?)-(\d{2}-\d{2}-\d{4})-to-(\d{2}-\d{2}-\d{4})\.csv$'

directory = '/Users/siddhanthmate/Desktop/AllFiles/CODE/WORK_CODE/fintech/DATA/spot_data'

def process_file(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            data = list(reader)
            if data:
                collection.insert_many(data)
                print(f"Inserted {len(data)} records from {os.path.basename(filepath)}")
            else:
                print(f"No data found in {os.path.basename(filepath)}")
    except Exception as e:
        print(f"Error processing {os.path.basename(filepath)}: {e}")

def main():
    files = os.listdir(directory)

    matching_files = [f for f in files if re.match(pattern, f)]

    threads = []

    for filename in matching_files:
        filepath = os.path.join(directory, filename)
        thread = threading.Thread(target=process_file, args=(filepath,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

if __name__ == '__main__':
    main()


Inserted 248 records from NIFTY BANK-01-01-2021-to-01-01-2022.csv
Inserted 248 records from NIFTY BANK-01-01-2022-to-01-01-2023.csv
Inserted 253 records from NIFTY BANK-01-01-2020-to-01-01-2021.csv
